In [1]:
import os
from os import path
import glob
import pandas as pd
import json
from collections import defaultdict

In [2]:
# perf_dir = "/home/shtoshni/Research/events/models/perf"
perf_dir = "/share/data/speech/shtoshni/research/events/models/perf"
slurm_id_list = ["6437008"]  #"6407936"

files = []

for slurm_id in slurm_id_list:
    files.extend(sorted(glob.glob(path.join(perf_dir, slurm_id + "*")), 
                                  key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1])))
    
# files = sorted(files, key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1]))
print(len(files))

48


In [3]:
model_dict_list = []
for file in files:
    model_dict_list.append(json.loads(open(file).read()))

In [4]:
def determine_varying_attributes(model_dict_list, ignore_attribs=['model_dir', 'train', 'test', 'dev', 'pretrained_mention_model',
                                                                  'slurm_id', 'best_model_dir', 'data_dir']):
    attrib_to_vals = defaultdict(set)
    for model_dict in model_dict_list:
        for attrib, val in model_dict.items():
            if attrib in ignore_attribs:
                continue
            else:
                attrib_to_vals[attrib].add(val)
    
    
    varying_attribs = []
    for attrib, vals in attrib_to_vals.items():
        if len(vals) > 1:
            varying_attribs.append(attrib)
            
    return varying_attribs

In [5]:
varying_attribs = determine_varying_attributes(model_dict_list)


In [6]:
perf_df = pd.DataFrame(columns=(varying_attribs))

for model_dict in model_dict_list:
    perf_dict = {}
    for attrib in varying_attribs:
        perf_dict[attrib] = model_dict[attrib]
            
    perf_dict['fscore'] = round(model_dict['dev']['fscore'], 1)
    perf_dict['test_f'] = round(model_dict['test']['fscore'], 1)
                
    perf_df = perf_df.append(perf_dict, ignore_index=True)

In [8]:
idx = perf_df.groupby(varying_attribs)['fscore'].idxmax()
dev_max = perf_df.iloc[idx]
dev_max

,rnn_size,use_mem_context,sample_invalid,event_subtype_loss_wt,seed,fscore,test_f
0,500,False,0.2,1.0,0,45.5,47.6
1,500,False,0.2,1.0,1,47.4,46.8
4,500,False,0.2,5.0,0,45.4,47.5
5,500,False,0.2,5.0,1,48.2,46.7
8,500,False,0.2,10.0,0,46.2,47.1
9,500,False,0.2,10.0,1,46.4,47.7
24,500,False,0.5,1.0,0,44.9,46.0
25,500,False,0.5,1.0,1,45.6,46.2
28,500,False,0.5,5.0,0,47.2,46.9
29,500,False,0.5,5.0,1,48.4,47.7


In [10]:
if 'seed' in varying_attribs:
    varying_attribs.remove('seed')
perf_df.groupby(varying_attribs)[['fscore', 'test_f']].mean()


fscore  test_f
rnn_size use_mem_context sample_invalid event_subtype_loss_wt                
500      False           0.2            1.0                     46.45   47.20
                                        5.0                     46.80   47.10
                                        10.0                    46.30   47.40
                         0.5            1.0                     45.25   46.10
                                        5.0                     47.80   47.30
                                        10.0                    47.50   47.35
         True            0.2            1.0                     47.20   46.50
                                        5.0                     47.40   46.95
                                        10.0                    46.90   46.20
                         0.5            1.0                     45.75   45.45
                                        5.0                     47.65   44.90
                                        10.0                    47.55   46.60
1000     False           0.2            1.0                     46.50   46.55
                                        5.0                     46.85   47.40
                                        10.0                    47.45   46.50
                         0.5            1.0                     46.70   46.90
                                        5.0                     47.55   46.45
                                        10.0                    46.45   46.00
         True            0.2            1.0                     47.30   45.75
                                        5.0                     46.15   46.50
                                        10.0                    47.25   45.75
                         0.5            1.0                     46.10   46.15
                                        5.0                     48.15   47.00
                                        10.0                    47.25   46.95

In [11]:
perf_df.groupby(['use_mem_context'])[['fscore', 'test_f']].mean()


,fscore,test_f
use_mem_context,,
False,46.800000,46.854167
True,47.054167,46.225000


In [12]:
perf_df.groupby(['event_subtype_loss_wt'])[['fscore', 'test_f']].mean()


,fscore,test_f
event_subtype_loss_wt,,
1.0,46.40625,46.32500
5.0,47.29375,46.70000
10.0,47.08125,46.59375


In [13]:
perf_df.groupby(['sample_invalid'])[['fscore', 'test_f']].mean()


,fscore,test_f
sample_invalid,,
0.2,46.879167,46.650000
0.5,46.975000,46.429167


In [14]:
perf_df.groupby(['rnn_size'])[['fscore', 'test_f']].mean()

,fscore,test_f
rnn_size,,
500,46.879167,46.587500
1000,46.975000,46.491667
